In [64]:
# Libraries
import tkinter as tk
from tkinter import messagebox
import serial  
import json
import os
import time

In [65]:
# Parametri e variabili globali
usbport = 'COM5'
arduino = None
movements = []

## Functions

In [66]:
# This 2 functions maps an input value (in range 0-100) to the corresponding value on the servo, accordingly to the 
# configuration on the json file
def mapping(thumb_big_value, thumb_little_value, index_finger_value, middle_finger_value, ringPinky_value, forearm_value):
    with open("config.json", "r") as json_file:
    #Load the contents of the JSON file into a Python dictionary
        data = json.load(json_file)
    
    #thumb - big servo
    thumb_big = data["thumb_big"]
    
    #thumb - little servo
    thumb_little = data["thumb_little"]
    
    #index_finger servo
    index_finger = data["index_finger"]

    #middle_finger servo
    middle_finger = data["middle_finger"]
 
    #ring_finger servo
    ring_pinky = data["ring_pinky"]
  
    #forearm servo
    forearm = data["forearm"]

    #All inputs
    input_values = [thumb_big_value, thumb_little_value, index_finger_value, middle_finger_value, ringPinky_value, forearm_value]
    #All range
    fingers_data = [thumb_big, thumb_little, index_finger, middle_finger, ring_pinky,forearm]
    fingers_data_mapped = []
    i = 0
    for single_finger in fingers_data:
        start = single_finger["range_from"]
        stop = single_finger["range_to"]
        #print(single_finger)
        fingers_data_mapped.append(calculus(input_values[i],start,stop))
        i = i+1
        
    return fingers_data_mapped
        
def calculus(val,start,stop):
    if start==0:
        return int((val/100)*stop)
    else:
        new_stop = stop - start
        return int(((val/100)*new_stop)+start)  

In [67]:
#******************Funzione per salvare un pacchetto o un movimento ***************************
#Struttura di salvataggio: thumb_big, thumb_little, index, middle, ringPinky, forearm
# movement deve essere una list di list (matrice)
def save_movement(name, movement):
    
    # Carica i dati esistenti dal file JSON se esiste
    # Se il file json non esiste, lo crea e ci aggiunge il nuovo movimento
    dati = {}
    if os.path.exists("movements.json"):
        #Se il file esiste, apro il file
        with open("movements.json", 'r') as file_json:
            dati = json.load(file_json)
            
        # Verifica se il nome è già presente nei dati
        if name in dati:
            print(f"Un movimento con il nome '{name}' esiste già. L'aggiunta verrà annullata")
        else:
            # Aggiungi la nuova matrice ai dati
            dati[name] = movement

            # Salva il dizionario aggiornato in formato JSON nel file specificato
            with open("movements.json", "w") as json_file:
                json.dump(dati, json_file)
                print(f"Movimento '{name}' aggiunto con successo al file")
    else:
        dati[name] = movement
        with open("movements.json", "w") as json_file:
            json.dump(dati, json_file)
            print(f"Movimento '{name}' aggiunto con successo al file")

In [68]:
# Funzione per la validazione dei campi di input della GUI
def on_validate(action, index, value_if_allowed, prior_value, text, validation_type, trigger_type, widget_name):
    if action == '1':  # Inserimento di un carattere
        if text in '0123456789.-+':
            try:
                # Converti il valore in un float
                float_value = float(value_if_allowed)

                # Verifica se il valore è compreso tra 0 e 100
                if 0 <= float_value <= 100:
                    return True
                else:
                    return False
            except ValueError:
                return False
        else:
            return False
    else:
        return True


In [69]:
# Funzione per il pulsante esegui movimento nel FRAME1 della GUI
def on_submit(gui_instance):
    values = []
    for entry in gui_instance.entry_list:
        try:
            valore_numerico = float(entry.get())
            values.append(valore_numerico)
        except ValueError:
            messagebox.showerror("Error", "Enter valid numeric values in all boxes")
            return

    #messagebox.showinfo("Successo", f"Hai inserito i valori numerici: {values}")
    
    #Controllo se la connessione sulla seriale è già aperta. Se non lo è, la apro
    global arduino 
    if arduino is None or not arduino.is_open:
        arduino = serial.Serial(usbport, baudrate=9600) 
        time.sleep(4)
    
    # Comunicazione ad arduino dei valori
    # Mapping: thumb big - thumb little - index - middle - ring&pinky - forearm
    fingers_data_mapped = mapping(values[0],values[1],values[2],values[3],values[4],values[5])
    print(fingers_data_mapped)
    # Send values to arduino
    arduino.write(fingers_data_mapped)
    print("Commands send\n")

In [119]:
# Funzione per il salvataggio dei movimenti in frame2
def on_save(gui_instance):
    # Funzione chiamata quando si preme il pulsante per aprire la finestra di input
    def on_ok():
        input_text = entry.get()
        if input_text:
            #salvataggio nel json
            save_movement(input_text,movements)
            messagebox.showinfo("Info", f"Hai inserito il movimento: {input_text}")
            input_window.destroy()
            #pulizia della lista globale
            movements.clear()
            
    # Creazione della finestra di input
    input_window = tk.Toplevel(gui_instance)
    input_window.title("Inserisci il nome del movimento")
    input_window.geometry("400x150")
    
    # Widget Entry per inserire il testo
    entry = tk.Entry(input_window)
    entry.pack(pady=10)

    # Pulsante "OK" per confermare l'input
    ok_button = tk.Button(input_window, text="OK", command=on_ok)
    ok_button.pack(pady=10)
    


# Funzione per i valori successivi in frame2    
def on_next(gui_instance):
    packet = []
    for entry in gui_instance.entry_list:
        try:
            value = float(entry.get())
            packet.append(value)
        except ValueError:
            messagebox.showerror("Error", "Enter valid numeric values in all boxes")
            return
    movements.append(packet)
    print(movements)
    

## GUI

In [120]:
class GUI(tk.Tk):
    def __init__(self):
        super().__init__()

        self.title("Hand movement GUI")
        self.geometry("500x500")

        self.columnconfigure(0, weight=1)  # Prima colonna
        self.columnconfigure(1, weight=1)  # Seconda colonna
        
        # Creazione dei frame
        self.frame1 = tk.Frame(self)
        self.frame2 = tk.Frame(self)
        self.frame3 = tk.Frame(self)
        
        # Configurazione dei frame
        self.configure_frame1()
        self.configure_frame2()
        self.configure_frame3()
        
        # Configurazione menù
        self.configure_menu()
        
        
        # Mostra il primo frame all'avvio
        self.show_frame(self.frame1)

    # *************************************** FRAME 1 CONFIGURATION ***************************************************
    def configure_frame1(self):
        
        title = tk.Label(self.frame1, text="Rapid movement",font="8")
        title.grid(row=0,column=1,pady=40,sticky="w")
        validate_cmd = self.frame1.register(on_validate)
        
        
        # Text1
        label1 = tk.Label(self.frame1, text="Thumb - big servo")
        label1.grid(row=1,column=0)
        thumb_big = tk.Entry(self.frame1, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_big.insert(0, "Value") 
        thumb_big.configure(justify=tk.CENTER) 
        thumb_big.grid(row=1,column=1)
        
        # Text2
        label2 = tk.Label(self.frame1, text="Thumb - little servo")
        label2.grid(row=2,column=0,padx=10)
        thumb_little = tk.Entry(self.frame1, fg='black',validate="key", 
                                validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_little.insert(0, "Value") 
        thumb_little.configure(justify=tk.CENTER) 
        thumb_little.grid(row=2,column=1,pady=5)

        
        # Text3
        label3 = tk.Label(self.frame1, text="Index finger")
        label3.grid(row=3,column=0,padx=10)
        index_finger = tk.Entry(self.frame1, fg='black',validate="key", validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        index_finger.insert(0, "Value") 
        index_finger.configure(justify=tk.CENTER) 
        index_finger.grid(row=3,column=1,pady=5)
        
        # Text4
        label4 = tk.Label(self.frame1, text="Middle finger")
        label4.grid(row=4,column=0)
        middle_finger = tk.Entry(self.frame1, fg='black',validate="key", 
                                 validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        middle_finger.insert(0, "Value") 
        middle_finger.configure(justify=tk.CENTER) 
        middle_finger.grid(row=4,column=1,pady=5)
        # Text5
        label5 = tk.Label(self.frame1, text="Ring and Pinky")
        label5.grid(row=5,column=0)
        ring_pinky = tk.Entry(self.frame1, fg='black',validate="key", 
                              validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        ring_pinky.insert(0, "Value") 
        ring_pinky.configure(justify=tk.CENTER) 
        ring_pinky.grid(row=5,column=1,pady=5)
        
        # Text6
        label6 = tk.Label(self.frame1, text="Forearm")
        label6.grid(row=6,column=0)
        forearm = tk.Entry(self.frame1, fg='black',validate="key", 
                           validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        forearm.insert(0, "Value") 
        forearm.configure(justify=tk.CENTER) 
        forearm.grid(row=6,column=1,pady=5)
        
        #Valori dei campi innestati nell'istanza self
        self.entry_list = [thumb_big, thumb_little, index_finger, middle_finger, ring_pinky,forearm]

        # Button
        button1 = tk.Button(self.frame1, text="Execute", command=lambda: on_submit(self))
        button1.grid(row=7,column=1)
        
        

    # **************************************** FRAME 2 CONFIGURATION *************************************************
    def configure_frame2(self):
        
        title = tk.Label(self.frame2, text="Create movement",font="8")
        title.grid(row=0,column=1,pady=40,sticky="w")
        validate_cmd = self.frame2.register(on_validate)
        
        
        # Thumb - big servo
        label1 = tk.Label(self.frame2, text="Thumb - big servo")
        label1.grid(row=1,column=0)
        thumb_big = tk.Entry(self.frame2, fg='black',validate="key", 
                             validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_big.insert(0, "Value") 
        thumb_big.configure(justify=tk.CENTER) 
        thumb_big.grid(row=1,column=1)
        
        # Thumb - little servo
        label2 = tk.Label(self.frame2, text="Thumb - little servo")
        label2.grid(row=2,column=0,padx=10)
        thumb_little = tk.Entry(self.frame2, fg='black',validate="key", 
                                validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        thumb_little.insert(0, "Value") 
        thumb_little.configure(justify=tk.CENTER) 
        thumb_little.grid(row=2,column=1,pady=5)

        
        # Index finger
        label3 = tk.Label(self.frame2, text="Index finger")
        label3.grid(row=3,column=0,padx=10)
        index_finger = tk.Entry(self.frame2, fg='black',validate="key", validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        index_finger.insert(0, "Value") 
        index_finger.configure(justify=tk.CENTER) 
        index_finger.grid(row=3,column=1,pady=5)
        
        # Middle finger
        label4 = tk.Label(self.frame2, text="Middle finger")
        label4.grid(row=4,column=0)
        middle_finger = tk.Entry(self.frame2, fg='black',validate="key", 
                                 validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        middle_finger.insert(0, "Value") 
        middle_finger.configure(justify=tk.CENTER) 
        middle_finger.grid(row=4,column=1,pady=5)
        
        # Ring adn pinky
        label5 = tk.Label(self.frame2, text="Ring and Pinky")
        label5.grid(row=5,column=0)
        ring_pinky = tk.Entry(self.frame2, fg='black',validate="key", 
                              validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        ring_pinky.insert(0, "Value") 
        ring_pinky.configure(justify=tk.CENTER) 
        ring_pinky.grid(row=5,column=1,pady=5)
        
        # Forearm
        label6 = tk.Label(self.frame2, text="Forearm")
        label6.grid(row=6,column=0)
        forearm = tk.Entry(self.frame2, fg='black',validate="key", 
                           validatecommand=(validate_cmd, "%d", "%i", "%P", "%s", "%S", "%v", "%V", "%W"))
        forearm.insert(0, "Value") 
        forearm.configure(justify=tk.CENTER) 
        forearm.grid(row=6,column=1,pady=5)
        
        #Valori dei campi innestati nell'istanza self
        self.entry_list = [thumb_big, thumb_little, index_finger, middle_finger, ring_pinky, forearm]

       
        # Buttons
        button1 = tk.Button(self.frame2, text="Save", command=lambda: on_save(self))
        button1.grid(row=7,column=0,sticky='e',pady=15)
        
        button2 = tk.Button(self.frame2, text="Next", command=lambda: on_next(self))
        button2.grid(row=7,column=2,sticky='w',pady=15)

    # *************************************** FRAME 3 CONFIGURATION **********************************************
    def configure_frame3(self):
        labell3 = tk.Label(self.frame3, text="Saved movements")
        labell3.pack(pady=5)

        
    # Funzione per configurare il menù della GUI 
    def configure_menu(self):
        # Creazione del menu
        menubar = tk.Menu(self)
        self.config(menu=menubar)
        frame_menu = tk.Menu(menubar, tearoff=0)
        frame_menu.add_command(label="Rapid movement", command=lambda: self.show_frame(self.frame1))
        frame_menu.add_command(label="Create movement", command=lambda: self.show_frame(self.frame2))
        frame_menu.add_command(label="Saved movements", command=lambda: self.show_frame(self.frame3))
        frame_menu.add_command(label="Shutdown", command=lambda: self.close())

        menubar.add_cascade(label="Menù", menu=frame_menu)

    def show_frame(self, frame):
        # Nasconde tutti i frame e mostra solo quello specificato
        self.frame1.grid_forget()
        self.frame2.grid_forget()
        self.frame3.grid_forget()
        frame.grid()
       
    #Chiusura programma e com. seriale
    def close(self):
        print("Sto chiudendo tutto")
        global arduino
        arduino.close()
        self.destroy() 

    

if __name__ == "__main__":
    app = GUI()
    app.mainloop()


[[1.0, 2.0, 3.0, 4.0, 44.0, 5.0]]
[[1.0, 2.0, 3.0, 4.0, 44.0, 5.0], [11.0, 22.0, 32.0, 42.0, 44.0, 52.0]]
Movimento 'movimento2' aggiunto con successo al file


In [121]:
movements


[]